In [1]:
import numpy as np

import qiskit
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_provider import IBMProvider
from qiskit_ionq import IonQProvider
from qiskit_aer.primitives import Sampler, Estimator

In [2]:
ideal_simulator = AerSimulator()

ibm_provider = IBMProvider()
noise_model = NoiseModel.from_backend(ibm_provider.get_backend('ibm_brisbane'))
ibm_noisy_simulator = AerSimulator(method='density_matrix', noise_model=noise_model)

noise_model_kyoto = NoiseModel.from_backend(ibm_provider.get_backend('ibm_kyoto'))
ibm_kyoto_simulator = AerSimulator(method='density_matrix', noise_model=noise_model_kyoto)

ionq_provider = IonQProvider()
ionq_simulator = ionq_provider.get_backend("ionq_simulator")
ionq_simulator.set_options(noise_model="harmony") 

In [3]:
# Create a basic Bell State circuit:
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

In [4]:

job = ideal_simulator.run(qc, shots=1024) 
print(job.result().get_counts())

{'00': 531, '11': 493}


In [5]:

job = ibm_noisy_simulator.run(qc, shots=1024) 
print(job.result().get_counts())

{'01': 25, '10': 27, '00': 470, '11': 502}


In [6]:

job = ibm_kyoto_simulator.run(qc, shots=1024) 
print(job.result().get_counts())

{'01': 107, '11': 409, '10': 110, '00': 398}


In [7]:
job = ionq_simulator.run(qc, shots=1024) 
print(job.result().get_counts())

{'00': 469, '01': 42, '10': 27, '11': 486}


In [8]:
ideal_sampler = Sampler()

In [9]:
job = ideal_sampler.run(qc)
print(job.result().quasi_dists)

[{3: 0.478515625, 0: 0.521484375}]


In [10]:
noisy_sampler = Sampler(backend_options={"noise_model": noise_model})

In [11]:
job = noisy_sampler.run(qc)
print(job.result().quasi_dists)

[{1: 0.0302734375, 2: 0.0166015625, 0: 0.5, 3: 0.453125}]


In [12]:
obs = qiskit.quantum_info.SparsePauliOp('ZZ')

In [13]:
ideal_estimator = Estimator()
job = ideal_estimator.run(qc, obs)
print(job.result())

EstimatorResult(values=array([1.]), metadata=[{'shots': 1024, 'variance': 0.0, 'simulator_metadata': [{'time_taken': 0.000904181, 'num_bind_params': 1, 'parallel_state_update': 16, 'parallel_shots': 1, 'required_memory_mb': 0, 'input_qubit_map': [[1, 1], [0, 0]], 'method': 'stabilizer', 'device': 'CPU', 'num_qubits': 2, 'sample_measure_time': 0.000263048, 'active_input_qubits': [0, 1], 'num_clbits': 2, 'remapped_qubits': False, 'runtime_parameter_bind': False, 'max_memory_mb': 32004, 'noise': 'ideal', 'measure_sampling': True, 'batched_shots_optimization': False, 'fusion': {'enabled': False}}]}])


In [14]:
noisy_estimator = Estimator(backend_options={"noise_model": noise_model})
job = noisy_estimator.run(qc, obs)
print(job.result())

EstimatorResult(values=array([0.8828125]), metadata=[{'shots': 1024, 'variance': 0.22064208984375, 'simulator_metadata': [{'time_taken': 0.001700787, 'num_bind_params': 1, 'parallel_state_update': 16, 'parallel_shots': 1, 'required_memory_mb': 1, 'input_qubit_map': [[1, 1], [0, 0]], 'method': 'density_matrix', 'device': 'CPU', 'num_qubits': 2, 'sample_measure_time': 0.000134114, 'active_input_qubits': [0, 1], 'num_clbits': 2, 'remapped_qubits': False, 'runtime_parameter_bind': False, 'max_memory_mb': 32004, 'noise': 'superop', 'measure_sampling': True, 'batched_shots_optimization': False, 'fusion': {'applied': False, 'max_fused_qubits': 2, 'threshold': 7, 'enabled': True}}]}])


In [15]:
noisy_estimator = Estimator(backend_options={"noise_model": noise_model_kyoto})
job = noisy_estimator.run(qc, obs)
print(job.result())

EstimatorResult(values=array([0.515625]), metadata=[{'shots': 1024, 'variance': 0.734130859375, 'simulator_metadata': [{'time_taken': 0.001768856, 'num_bind_params': 1, 'parallel_state_update': 16, 'parallel_shots': 1, 'required_memory_mb': 1, 'input_qubit_map': [[1, 1], [0, 0]], 'method': 'density_matrix', 'device': 'CPU', 'num_qubits': 2, 'sample_measure_time': 0.000134033, 'active_input_qubits': [0, 1], 'num_clbits': 2, 'remapped_qubits': False, 'runtime_parameter_bind': False, 'max_memory_mb': 32004, 'noise': 'superop', 'measure_sampling': True, 'batched_shots_optimization': False, 'fusion': {'applied': False, 'max_fused_qubits': 2, 'threshold': 7, 'enabled': True}}]}])
